In [ ]:
# Initial EDA for a feel of the data.
# ---------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# 1. Loading the cleaned data.
clean_path = Path("../data/processed/clean_hdb.csv")  # Path may need adjustment.
df = pd.read_csv(clean_path, parse_dates=["sale_date"])

# 2. Histograms for resale_price & price_per_sqm.
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
df["resale_price"].plot(kind="hist", bins=50, ax=ax[0])
ax[0].set_title("Resale Price (SGD)")
ax[0].set_xlabel("SGD")

df["price_per_sqm"].plot(kind="hist", bins=50, ax=ax[1])
ax[1].set_title("Price per sqm (SGD)")
ax[1].set_xlabel("SGD")
plt.tight_layout()
plt.show()

'''
First quick sanity-check on the data from these histograms:

1. Resale Price (left):
   * Classic right-skew. Most deals are between $200-500k, with a long tail of million-dollar units.
   * No weird spikes at zero or negative values, so data load and outlier filter worked.

2. Price per sqm (right):
   * Also right-skewed, peaking around SGD 2-4k/m².
   * Values under $1k/m² are rare but still plausible for old leases or fringe towns.
   * No second mode, so a log-transform might help linear models later. Tree models should be fine with the raw distribution.

These shapes look right, which confirms the CSV download and cleaning worked as expected.
'''


In [ ]:
# 3. Checking the yearly trend by flat_type, but only for 3-, 4-, and 5-room flats to keep it clean.
keep_types = ["3 room", "4 room", "5 room"]
subset = df[df["flat_type"].str.lower().isin(keep_types)]
pivot  = subset.pivot_table(index="sale_year",
                            columns="flat_type",
                            values="resale_price",
                            aggfunc="median")

pivot.plot(figsize=(10, 5))
plt.title("Median Resale Price by Flat Type")
plt.ylabel("SGD")
plt.xlabel("Year")
plt.grid(True, alpha=0.3)
plt.show()

'''
Key observations from the line plot of median resale prices:

• The size premium is always there. 5-room flats are always above 4-room, and 4-room are always above 3-room. The lines never cross, which is a good sign for one-hot encoding.

• The big market cycles are visible:
  - The '96-97 peak and the Asian Financial Crisis cooldown.
  - The surge from 2009-2013 after the GFC.
  - The 2013-2019 plateau due to government cooling measures.
  - The 2020-2025 spike from the Covid supply squeeze.

• The price gap widens after 2019. Bigger flats appreciated faster recently, which the model should be able to capture.

Nothing looks weird or impossible here, so the `flat_type` parsing and year aggregation seem fine.
'''

In [ ]:
# 4. Boxplot of price_per_sqm by town for the top 6 towns by volume.
top_towns = (df["town"].value_counts().head(6).index)
plt.figure(figsize=(10, 5))
df[df["town"].isin(top_towns)] \
    .boxplot(column="price_per_sqm", by="town", rot=45)
plt.title("Price per sqm – Top 6 Towns")
plt.suptitle("")
plt.ylabel("SGD")
plt.show()

'''
Observations:
• The median lines confirm the usual pecking order: mature towns like Ang Mo Kio and Bedok beat the outer towns like Woodlands and Yishun.
• The tall whiskers and all the dots show a healthy resale volume and some expected outliers, probably new premium projects.
Takeaway is that the town dummy features will be important, and nothing looks suspicious here.
'''

In [ ]:
# Checking the correlation after stripping out location and year effects.
# ---------------------------------------------------------------

# 1) Choosing a local 'neighbourhood' based on town and sale_year.
grp_cols = ["town", "sale_year"]

# 2) These are the numeric features to study.
num_cols = [
    "resale_price",
    "price_per_sqm",
    "floor_area_sqm",
    "lease_remaining_years",
    "flat_age",
]

# 3) Demeaning every numeric column within each town-year bucket.
#    The residuals will be 'this flat's value minus the average for its town & year'.
resid = (
    df.groupby(grp_cols)[num_cols]           # Splitting by town × year.
      .transform(lambda x: x - x.mean())     # And subtracting the local mean.
)

# 4) Making a correlation matrix on those residuals.
corr = resid.corr()

# 5) And a heat-map to visualize it.
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 5))
plt.imshow(corr, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar()

ticks = range(len(num_cols))
plt.xticks(ticks, num_cols, rotation=45, ha="right")
plt.yticks(ticks, num_cols)
plt.title("Correlation (demeaned by town × year)")

# Overlaying the exact r values to be clear.
for i in ticks:
    for j in ticks:
        plt.text(j, i, f"{corr.iat[i, j]:.2f}",
                 ha="center", va="center", fontsize=8)

plt.tight_layout()
plt.show()

# Quick sanity check, looking at a single recent year.
# ---------------------------------------------------------------
recent = df[df.sale_year >= 2023]             # Just looking at 2023–24.
corr_recent = recent[["price_per_sqm",
                      "flat_age",
                      "lease_remaining_years"]].corr()
print(corr_recent)
# This confirms that price_per_sqm goes down with age and up with remaining lease.

'''
Thoughts on the correlation after removing town and year bias:
After subtracting each flat’s town-year mean, a couple of things popped out:

•  The lease signal is clear now:
   - `price_per_sqm` goes up with `lease_remaining_years` (+0.61)
   - `price_per_sqm` goes down with `flat_age` (-0.61)

•  Floor area drives the absolute price, not the price per square meter.
   `floor_area_sqm` has a 0.82 correlation with `resale_price`, but only 0.21 with ppsm.

By stripping out location and inflation, the real physical drivers that the model needs to learn have been isolated.
'''

In [ ]:
# 6. Checking the lease-decay curve (99 yrs on the left → 0 yrs on the right).
(
    df.groupby("lease_remaining_years")["price_per_sqm"]
      .median()
      .sort_index(ascending=False)      # Showing it from 99 down to 0 years.
      .plot(figsize=(6, 4))
)

plt.title("Lease Remaining vs Price per sqm")
plt.xlabel("years of lease left")
plt.ylabel("median SGD per sqm")
plt.grid(alpha=0.3)
plt.show()


# Picking a single, 'average' suburb to see the lease decay impact.
# Choosing Tampines for this.
subset = df[df["town"] == "TAMPINES"]      
decay = (
    subset.groupby("lease_remaining_years")["price_per_sqm"]
           .median()
           .sort_index()                       # And here going from 0 to 99 years.
)

decay.plot(figsize=(6,4))
plt.title("Lease-decay in Tampines")
plt.xlabel("years of lease left")
plt.ylabel("median SGD per sqm")
plt.grid(alpha=.3)
plt.show()

'''
Sanity check on the lease-decay curves:
1. Island-wide (excluding prime):
   After dropping the super-premium towns, the curve looks right: more lease left means a higher median price per square meter. The little uptick at the 95+ year mark is just a few new launches.
2. Single suburb – Tampines:
   Here in one location, the pattern is really clear. Flats with about 60 years left trade for over $6k/m², and it slides down to about $2.5k/m² when the lease is under 80 years.
Both of these lines confirm the 99-year lease decay that HDB publishes, so the `lease_remaining_years` feature is working and ready for the model.
'''

In [ ]:
# 7. Bar chart of the top towns for the latest year.
latest = df[df.sale_year == df.sale_year.max()]
top10  = (latest.groupby("town")["price_per_sqm"]
                 .median()
                 .sort_values(ascending=False)
                 .head(10))
top10.plot(kind="barh")

'''
Thoughts on the 2024 median price per square meter for the top 10 towns:
Queenstown is still king of the resale market. The usual suspects (Central Area, Bukit Timah, Bukit Merah) are right behind. Serangoon is at the bottom of the list at around SGD 7k/m². This pattern validates the town-dummy encoding and confirms there are no obvious price glitches.
'''

In [ ]:
''' Commentary:

Key signals:
	•	Lease is king. After stripping out the town and year effects, the price/sqm drops by about 30% between a 95-year and a 70-year lease.
	•	Size matters, but just for the total price. Floor area has a strong +0.8 correlation with resale_price, but almost none with price/sqm. Bigger flats cost more, but not at a premium rate.
	•	The location premium is real. Queenstown, Central Area, and Bukit Timah are all >= SGD 9k/sqm, while Woodlands & Yishun are closer to 2–3k/sqm. Dummy-encoding the towns is a must.
	•	Macro swings are clear. There are peaks in '96-97 and 2013, and a long flat stretch from 2013-19, then a Covid spike.
	•	The flat-type gap widens after 2020. 5-room units now have about a 40% median premium over 3-room flats.

Data gaps noticed:
	•	No block-level coordinates, MRT distance, or the floor level, so the location is a bit coarse.
	•	Missing the interior condition and any renovations, so some of the price variance is just noise that can't be reduced.
	•	Lease remaining is just in years, not months.
	•	Ignoring historical macro variables for now.

Bottom line: the dataset is clean, the patterns make sense, and my key drivers are ready for modelling. Time for Phase 3, the baseline model.
'''